<a href="https://colab.research.google.com/github/mbohling/spiking-neuron-model/blob/main/Hodgkin-Huxley/SpikingNeuronModel_HH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The Spiking Neuron Model - Coding Challenge Problems (Part 3)


# Hodgkin-Huxley Spiking Neuron Model
This interactive document is meant to be followed as the reader makes their way through chapter: *The Spiking Neuron Model*.  Each model presented in the chapter will have a section consisting of a step-by-step walkthrough of a simple Python implementation.  This is followed by an interface to run simulations with different parameter values to answer the Coding Challenge Problems.

For each model covered in the chapter, there is a section called **Coding Challenge Problems.**  This is where you will find user-interface components such as value sliders for various parameters.  Use these controls to answer the questions from the text.


**Content creators**
*  Maxwell E. Bohling
*  Lawrence C. Udeigwe, Ph.D

## How It Works
Google Colab Notebooks have both *Content* cells and *Code* cells. As you progress through the notebook, you MUST make sure to run each code cell as you come to them.  Otherwise, you may run into errors when executing a code cell. Each code cell has a Play button next to it which will execute the code. (Some code may be hidden by default. This is generally because the code is more complex and is not necessary to understand in order to complete the model implementations or to answer the chapter Coding Challenge Problems).

**IMPORTANT**: You have been provided a link to view a **copy** of the original notebooks. You will find that you can edit the content of any cell. If you accidently change a cell, such as a line of code and/or run into errors as you try to run subsequent blocks, simply refresh the page, OR go to the *Runtime menu* and select *Restart runtime*. It is also suggested that you go to the *Edit menu* and select *Clear all outputs*. This will always allow you to revert the notebook to the original version (though you will have to run each code block again.)

For each model covered in the chapter, there is a section called **Coding Challenge Problems**. This is where you will find user-interface components such as value sliders for various parameters. Use these controls to answer the questions from the text.


 Execute the code block. **Initialize Setup**

In [ ]:
#@title Initialize Setup
#@markdown (No need to understand this code, simply make sure you run this first).
import sys
import functools as ft
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import scipy as sc

# [BLOCK TAG: INIT]

try:
  blockSet = [ ]
except:
  print('Something went wrong! Try Refreshing the page.')

blockTags = ['INIT','VP1','NP1','SS1','SS2','SS3','CS1','CS2','CS3','VR1']

def pushBlockStack(tag):
    if tag in blockSet:
        return 1
    indx = blockTags.index(tag)
    if len(blockSet) != indx:
        print('ERROR: BLOCK TAG:',tag,'executed out of sequence. Missing BLOCK TAG:', blockTags[indx-1])
        return 0
    else:
        blockSet.append(tag)
        return 1

def printError():
  message = 'Something went wrong!\n\n'  
  message = message + 'Check for the following:\n\n'
  message = message + '\t1.  All previous code blocks have been run the order they appear and output a success message.\n'
  message = message + '\t2.  No other code has been altered.\n\n'
  message = message + 'and then try running the code block again.'
  message = message + ' If there is still an error when executing the code block, try the following:\n\n'
  message = message + '\t1.  Go to the \'Runtime\' menu and select \'Restart Runtime\', then in the \'Edit\' menu, select \'Clear all outputs\'.\n'
  message = message + '\t2.  Refresh the page.\n\n'
  message = message + 'and be sure to run each of the previous code blocks again beginning with \'Initialize Setup\'.\n'
  print(message)
  return 0

def printSuccess(block):
  success = 0
  if len(block) == 0 or pushBlockStack(block) != 0:
    message = 'Success!  Move on to the next section.'
    print(message)
    success = 1
  return success

def checkVoltageParameters(Vrest):
  print('Checking Voltage Parameters... ')
  try:
      check_Vrest = Vrest
  except:
    return 0
  else:
    vals = [Vrest]
    correct_vals = [-70]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False): 
      return 0
  return 1
 
def checkNeuronProperties(A, Ie, GL, GK, GNa, EL, EK, ENa):
  print('Checking Neuron Properties... ')
  try:
      check_A = A
      check_Ie = Ie
      check_GL = GL
      check_GK = GK
      check_GNa = GNa
      check_EL = EL
      check_EK = EK
      check_ENa = ENa
  except:
    return 0
  else:
    vals = [A, Ie, GL, GK, GNa, EL, EK, ENa]
    correct_vals = [0.1, 1.00, 0.03, 3.6, 12, -70, -77, 55]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False): 
      return 0
  return 1

def checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, n_initial, m_initial, h_initial, start_current, end_current):
  print('Checking Simulation Setup... ')
  try:
    check_Vrest = Vrest
    check_Vinitial = Vinitial
    check_t0 = t0
    check_dt = dt
    check_t_final = t_final
    check_time = time
    check_n_initial = n_initial
    check_m_initial = m_initial
    check_h_initial = h_initial
    check_start_current = start_current
    check_end_current = end_current
  except:
    return 0
  else:
    vals = [Vrest, Vinitial, EL, t0, dt, t_final, time, n_initial, m_initial, h_initial, start_current, end_current]
    correct_vals = [-70, -70, 0, 0.01, 20, 0.1399, 0.0498,  0.6225, 1, 2]
    if ft.reduce(lambda i, j : i and j, map(lambda m, k: m == k, vals, correct_vals),  False):  
      if len(time) != 2000 or time[0] != 0 or time[-1] != 20:
        return 0
  return 1

def checkValues():
  chk = 3
  if checkVoltageParameters(Vrest) < 1:
    print('FAIL\n')
    chk = chk - 1
  else:
    print('PASS\n')
  if checkNeuronProperties(A, Ie, GL, GK, GNa, EL, EK, ENa) < 1:
    print('FAIL\n')
    chk = chk - 1  
  else:
    print('PASS\n')
  if checkSimulationSetup(Vrest, Vinitial, EL, t0, dt, t_final, time, n_initial, m_initial, h_initial, start_current, end_current) < 1:
    print('FAIL\n')
    chk = chk - 1
  else:
    print('PASS\n')
  return chk

try:
  check_sys = sys
except:
  printError()
else:
  modulename = 'functools'
  if modulename not in sys.modules:
    printError()
  else:
    printSuccess('INIT')

## Walkthrough
The goal of this section is to write a Python implementation of the Hodgkin-Huxley model.  Recall from the chapter text that we need to account for both activation and inactivation gating variables in order to simulate the persistent and transient conductances involved in the membrane current equation.

### Membrane Current
The Hodgkin-Huxley model is expressed as membrane current equation given as:

> $ \displaystyle i_{m} = \overline{g}_{L}(V-E_{L}) + \overline{g}_{K}n^4(V-E_{K}) + \overline{g}_{Na}m^3h(V-E_{Na})$

with maximal conductances $\overline{g}_{L},\;$ $\overline{g}_{K}\;$ $\overline{g}_{Na}\;$ and reversal potentials $E_{L},\;$ $E_{K},\;$ $E_{Na}$.

As with the previous models, Euler's method is used to compute the time evolution of the membrane potential $V$.  For this model, we use the same numerical integration method to compute the evolution of the gating variables $n$, $m$, and $h$. 

### Membrane Equation
Recall that the membrane equation is expressed as follows:

> $ \displaystyle \frac{dV}{dt} = -i_m+ \frac{I_{e}}{A} $

### Voltage Parameters

As opposed to the integrate-and-fire model, the Hodgkin-Huxley model does not utilize a spiking mechanism.  Therefore, we only need to define the *voltage parameter* that determines the *resting* membrane potential value.

*   $ V_{rest} = -70\;$*mV*




In [ ]:
# [BLOCK TAG: VP1]

try:
  check_BlockSet = blockSet
except:
  print('ERROR: BLOCK TAG: VP1 executed out of sequence. Missing BLOCK TAG: INIT')
else:
  try:
    ##################################################################################
    # Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
    Vrest = -70
    ##################################################################################
  except:
    printError()
  else:
    printSuccess('VP1')

### Neuron Properties
The membrane equation is described by a total membrane current $i_{m}$ as a sum of:

1.  A *leakage current*: $ \displaystyle\; \overline{g}_{L}(V-E_{L}) $
2.  A *persistent current*:  $\displaystyle\; \overline{g}_{K}n^4(V-E_{K}) $
3.  A *transient current*:  $\displaystyle\; \overline{g}_{Na}m^3h(V-E_{Na})$

Thus, the persistent conductance is modeled as a K$^+$ conductance and the transient conductance is modeled as a Na$^+$ conductance.  For each current, we define the maximimal conductances:

*   $ \displaystyle\; \overline{g}_{L} = 0.3\;$mS / mm$^2$
*   $ \displaystyle\; \overline{g}_{K} = 35\;$mS / mm$^2$
*   $ \displaystyle\; \overline{g}_{Na} = 40\;$mS / mm$^2$

and reversal potentials:

*   $ \displaystyle\; E_{L} = -70\;$mV
*   $ \displaystyle\; E_{K} = -77\;$mV
*   $ \displaystyle\; E_{Na} = 55\;$mV

Lastly, as seen in the membrane equation for the model, we must define the value of the injected current, and the neuronal surface area:

*   $ \displaystyle\; I_{e} = 1\;$nA
*   $ \displaystyle\; A = 0.1\;$mm$^2$

In [ ]:
# [BLOCK TAG: NP1]

try:
  ##################################################################################
  #Maximal Conductances - Units nS/mm^2
  GL = 0.03
  GK = 3.6
  GNa = 12

  # Reversal Potentials - Units mV
  EL = -70
  EK = -77
  ENa = 55

  # Input current: Ie - Units nA (1 nA = 10-9 Amperes)
  Ie = 1.00

  # Neuron Surface Area - Units mm^2
  A = 0.1
  ##################################################################################
except:
  printError()
else:
  printSuccess('NP1')

### Simulation Setup
To setup our simulation, we need initial values of each variable: $V$, $n$, $m$, and $h$ as well as a list to hold the values over time.

Set initial values as:

* $V_{initial}= V_{rest} = -70\;$*mV*
* $n_{initial} = 0.1399$
* $m_{initial} = 0.0498$
* $h_{initial} = 0.6225$

With each value defined at time $t = 0$, let $V_0 = V_{initial}, n_0 = n_{initial},  m_0 = m_{initial}, h_0 = h_{initial} $.  

The initial membrane current is then:

*  $\displaystyle i_{initial} = \overline{g}_{L}(V_0-E_{L}) + \overline{g}_{K}n_0^4(V_0-E_{K}) + \overline{g}_{Na}m_0^3h_0(V_0-E_{Na})$ 

Here we make use of the **numpy** library (to learn more about how to use this library, go to  https://numpy.org/doc/stable/).

In [ ]:
# [BLOCK TAG: SS1]

try:
  ##################################################################################
  # Initial voltage
  Vinitial = Vrest

  # Initial gating variable values (Probability [0, 1])
  n_initial = 0.1399
  m_initial = 0.0498
  h_initial = 0.6225

  # Initial membrane current 
  im_initial = GL*(Vinitial-EL)+GK*np.power(n_initial,4)*(Vinitial-EK)+GNa*np.power(m_initial,3)*h_initial*(Vinitial-ENa)
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS1')

We will be running a 20 ms simulation.  The following lines of code setup a time span for the simulation.  This is simply a matter of defining the start time $t_{0} = 0$ and the total length (in ms) of the simulation: $t_{final} = 20$.  

Throughout the simulation, we calculate the membrane potential $V$ at each *time-step*.  The time-step is the change in time for each iteration of the simulation, for example if $t_{0} = 0$, the next computation of $V$ is performed at $t_{0} + dt$.  

Thus, by setting $dt = 0.01$ (in ms), the simulation will compute $V$, $n$, $m$, and $h$ at time $t = 1, 2, \ldots, t_{final}$. 

In [ ]:
# [BLOCK TAG: SS2]

try:
  ##################################################################################
  # Simulation Time Span (0 to 20ms, dt = 0.01ms)
  t0 = 0
  dt = 0.01
  t_final = 20

  # What does the linspace() function do?
  time = np.linspace(t0, t_final,  2000)
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS2')

Next, we define the time $t$ at which the injected current $I_{e}$ is *switched on* and applied to the neuron, and the time $t$ at which the injected current is *switched off*.

For the Hodgkin-Huxley model, we run a shorter simulation and we apply the current from $t = 1\;$ms to $t = 2\;$ms.

In [ ]:
# [BLOCK TAG: SS3]

try:
  ##################################################################################
  # Time at which the current is applied - Units ms
  start_current = 1
  
  # Time at which the current is switched off - Units ms
  end_current = 2
  ##################################################################################
except:
  printError()
else:
  printSuccess('SS3')

### Computing and Storing $\frac{dV}{dt}$, $\frac{dn}{dt}$, $\frac{dm}{dt}$, $\frac{dh}{dt}$

We are about ready to finish the code implementation for simulating a Hodgkin-Huxley model neuron.

We need some way to store the values of the membrane potential $V, n, m, h$ at each time step.  To do this, we simply create empty lists $V[t], n[t], m[t], h[t]$ with a length equal to the number of time-steps of our simulation.

In [ ]:
# [BLOCK TAG: CS1]

try:
  ##################################################################################
  # Create a list V(t) to store the value of V at each time-step dt
  V = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value Vinitial
  V[0] =  Vinitial

  # Create lists to store the value of each gating variable at each time-step dt
  n = [0] * len(time)
  m= [0] * len(time)
  h = [0] * len(time)

  # Set the initial value at time t = t0 to the initial values
  n[0] =  n_initial
  m[0] =  m_initial
  h[0] =  h_initial

  # Create list to store value of membrane current at each time-step dt
  im = [0] * len(time)

  # Set the initial value at time t = t0 to the initial value im_initial
  im[0] = im_initial
  ##################################################################################
except:
  printError()
else:
  printSuccess('CS1')

### Opening and Closing Rate Functions for Gating Variables

The gating variables $n$, $m$, and $h$ represent **probabilities** that a gate mechanism in both the persistent and transient ion-conducting channels are open or *activated*.   

For any arbitrary gating variable $z$,  the open probability of a channel at any time $ t $ is computed using  an *opening* rate function $\alpha_{z}(V)$ and a *closing* rate $\beta_{z}(V)$, both of which are functions of the membrane potential $V$.

Each gating variable is numerically integrated using Euler's method throughout the simulation, where for any arbitrary gating variable $z$, the rate functions are given as follows:

> $ \displaystyle \tau_{z}(V)\frac{dz}{dt} = z_{\infty}(V) - z $

where

>  $ \displaystyle \tau_{z}(V) = \frac{1}{\alpha_{z}(V) + \beta_{z}(V)} $

and

> $ \displaystyle z_{\infty}(V) = \frac{\alpha_{z}(V) }{\alpha_{z}(V) + \beta_{z}(V)} $




#### Fitted Rate Functions
Hodgkin and Huxley had fit the opening and closing rate functions using experimental data.  These are given as follows:

---
For activation variable $n$

> $ \displaystyle \alpha_{n}(V) = \frac{0.01(V+60)}{ 1 - \exp(-0.1(V+60))} $

> $ \displaystyle \beta_{n}(V) = 0.125\exp(-0.0125(V+70)) $

---
For activation variable $m$

> $ \displaystyle \alpha_{m}(V) = \frac{0.1(V+45)}{1 - \exp(-0.1(V+45))}$

> $ \displaystyle \beta_{m}(V) = 4\exp(-0.0556(V+70)) $

---
For inactivation variable $h$

> $ \displaystyle \alpha_{h}(V) = .07\exp(-0.05(V+70))$

> $ \displaystyle \beta_{h}(V) = \frac{1}{1 + \exp(-0.1(V+40))} $

We define separate functions for each gating variable.  These will take the membrane potential $V(t)$ as input, and ouput $dz$ where $z = n, m, h $.  

Using the functional forms and fitted rate functions, these functions compute the changes dn, dm, and dh at each time-step dt which depend on the membrane potential V at time t.


 Execute the code block. **Initialize Helper Functions**

In [ ]:
#@title Initialize Helper Functions
#@markdown (Double-Click the cell to show the code.)
# [BLOCK TAG: CS2]

##################################################################################
# Function: compute_dn
def compute_dn(v, n):
  alpha_n = (0.01*(v + 60))/(1 - np.exp(-0.1*(v+60)))
  beta_n = 0.125*np.exp(-0.0125*(v+70))

  n_inf = alpha_n/(alpha_n + beta_n)
  tau_n = 1/(alpha_n + beta_n)

  dn = (dt/tau_n)*(n_inf - n)
  return dn

# Function: compute_dm
def compute_dm(v, m):
  alpha_m = (0.1*(v + 45))/(1 - np.exp(-0.1*(v+45)))
  beta_m = 4*np.exp(-0.0556*(v+70))

  m_inf = alpha_m/(alpha_m + beta_m)
  tau_m = 1/(alpha_m + beta_m)

  dm = (dt/tau_m)*(m_inf - m)
  return dm

# Function: compute_dh
def compute_dh(v, h):
  alpha_h = 0.07*np.exp(-0.05*(v+70))
  beta_h =  1/(1 + np.exp(-0.1*(v+40)))

  h_inf = alpha_h/(alpha_h + beta_h)
  tau_h = 1/(alpha_h + beta_h)

  dh = (dt/tau_h)*(h_inf - h)
  return dh
##################################################################################

x = printSuccess('CS2')

Finally, we run our simulation according to the updated *pseudocode*

---

*for each time-step from $t = t_{0}$ to $t = t_{final}$*
> *If the current time $t \geq start_{current}\ $ and $\ t \leq end_{current}$*
>> $I_{e} = 1\;$nA

> *otherwise*
>> $I_{e} = 0\;$nA

> *First compute the open probabilites for each gating variable*

> $ \displaystyle dn = $ **compute_dn**$(V[t], n[t])$

> *Update* $ n[t+1] = n[t] + dn $

> $ \displaystyle dm = $ **compute_dm**$(V[t], m[t])$

> *Update* $ m[t+1] = m[t] + dm $

> $ \displaystyle dh = $ **compute_dh**$(V[t], h[t])$

> *Update* $ h[t+1] = h[t] + dh $

> $ \displaystyle i_{m}[t+1] = \overline{g}_{L}(V[t]-E_{L}) + \overline{g}_{K}n[t+1]^4(V[t]-E_{K}) + \overline{g}_{Na}m[t+1]^3h[t+1](V[t]-E_{Na})$

> *Use Euler's Method of Numerical Integration*

> $ \displaystyle dV= dt\left(-i_m[t+1]+ \frac{I_{e}}{A}\right) $

> *Update* $V[t+1] = V[t] + dV$


*end*

---

This translates to the following Python code.

In [ ]:
# [BLOCK TAG: CS3]

try:
  chk = checkValues()
except:
  printError()
else:
  try:
    ##################################################################################
    # For each timestep we compute V and store the value
    for t in range(len(time)-1):

      # If time t >= 1 ms and t <= 2 ms, switch Injected Current ON
      if time[t] >= start_current and time[t]  <= end_current:
        ie = Ie
      # Otherwise, switch Injected Current OFF
      else:
        ie = 0

      # For each timestep we compute n, m and h and store the value
      dn = compute_dn(V[t], n[t])
      n[t+1] = n[t] + dn

      dm = compute_dm(V[t], m[t])
      m[t+1] = m[t] + dm

      dh = compute_dh(V[t], h[t])
      h[t+1] = h[t] + dh

      # Use these values to compute the updated membrane current
      im[t+1] = GL*(V[t]-EL)+GK*np.power(n[t+1],4)*(V[t]-EK)+GNa*np.power(m[t+1],3)*h[t+1]*(V[t]-ENa)

      # Using Euler's Method for Numerical Integration (See Chapter Text)
      # we compute the change in voltage dV as follows (using the model equation)
      dV = dt*(-1*im[t+1] + ie/A)

      # Store this new value into our list
      V[t+1] = V[t] + dV
      ##################################################################################     
  except:
    printError()
  else:
    if chk == 3:
      printSuccess('CS3')
    else:
      printError()

### Visualizing Results
Now we have values of $V$, $i_m$, $n$, $m$, and $h$ for each time-step of the simulation, we can visualize the results by using Python to plot the data.   This makes use of another widely used library **plotly** (to learn more about plotting data with this library, go to https://plotly.com/python/reference/index/).

In [ ]:
# [BLOCK TAG: VR1]

try:
  if 'CS2' not in blockSet:
    print('ERROR: BLOCK TAG: VR1 executed out of sequence. Missing BLOCK TAG: CS3')
  else:
    try:
      ##################################################################################
      # Data 
      x = list(time[0:-2])
      y = V

      # Plot data
      fig = make_subplots(
          rows=3, cols=1,  shared_xaxes = True, vertical_spacing=0.1, 
          subplot_titles=('V over Time', 'i_m over Time', 'n, m, h over Time')
      )

      # Add traces
      fig.add_trace(go.Scatter(name='V', x=x, y=y), row=1, col=1)
      fig.add_trace(go.Scatter(name='i_m', x=x, y=im), row=2, col=1)
      fig.add_trace(go.Scatter(name='n', x=x, y=n), row=3, col=1)
      fig.add_trace(go.Scatter(name='m', x=x, y=m), row=3, col=1)
      fig.add_trace(go.Scatter(name='h', x=x, y=h), row=3, col=1)

      # Update xaxis properties
      fig.update_xaxes(title_text="Time t (ms)", row=3, col=1)

      # Update yaxis properties
      fig.update_yaxes(title_text="Membrane Potential V (mV)", row=1, col=1)
      fig.update_yaxes(title_text="Membrane Current i_m", row=2, col=1)
      fig.update_yaxes(title_text="n, m, h (Probability)", row=3, col=1)

      # Update title and size
      fig.update_layout(height=800, width=700, 
                                      title_text='Hodgkin-Huxley Model Neuron',
                                      showlegend = True)

      # Update theme
      fig.layout.template = 'plotly_dark'

      # Show figure
      fig.show()
      ##################################################################################

      printSuccess('VR1')
    except:
      printError()
except:
  printError()

## Hodgkin-Huxley Spiking Neuron Model - Full Code

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70

#Maximal Conductances - Units nS/mm^2
GL = 0.03
GK = 3.6
GNa = 12

# Reversal Potentials - Units mV
EL = -70
EK = -77
ENa = 55

# Input current: Ie - Units nA (1 nA = 10-9 Amperes)
Ie = 1.00

# Neuron Surface Area - Units mm^2
A = 0.1

# Simulation Time Span (0 to 20ms, dt = 0.01ms)
t0 = 0
dt = 0.01
t_final = 20
time = np.linspace(t0, t_final,  2000)

# Time at which the current is applied - Units ms
start_current = 1

# Time at which the current is switched off - Units ms
end_current = 2

# Initial voltage
Vinitial = Vrest

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the initial value Vinitial
V[0] =  Vinitial

# Initial gating variable values (Probability [0, 1])
n_initial = 0.1399
m_initial = 0.0498
h_initial = 0.6225

# Create lists to store the value of each gating variable at each time-step dt
n = [0] * len(time)
m= [0] * len(time)
h = [0] * len(time)

# Set the initial value at time t = t0 to the initial values
n[0] =  n_initial
m[0] =  m_initial
h[0] =  h_initial

# Initial membrane current 
im_initial = GL*(V[0]-EL)+GK*np.power(n[0],4)*(V[0]-EK)+GNa*np.power(m[0],3)*h[0]*(V[0]-ENa)

# Create list to store value of membrane current at each time-step dt
im = [0] * len(time)

# Set the initial value at time t = t0 to the initial value im_initial
im[0] = im_initial

# Function: compute_dn
def compute_dn(v, n):
  alpha_n = (0.01*(v + 60))/(1 - np.exp(-0.1*(v+60)))
  beta_n = 0.125*np.exp(-0.0125*(v+70))

  n_inf = alpha_n/(alpha_n + beta_n)
  tau_n = 1/(alpha_n + beta_n)

  dn = (dt/tau_n)*(n_inf - n)
  return dn

# Function: compute_dm
def compute_dm(v, m):
  alpha_m = (0.1*(v + 45))/(1 - np.exp(-0.1*(v+45)))
  beta_m = 4*np.exp(-0.0556*(v+70))

  m_inf = alpha_m/(alpha_m + beta_m)
  tau_m = 1/(alpha_m + beta_m)

  dm = (dt/tau_m)*(m_inf - m)
  return dm

# Function: compute_dh
def compute_dh(v, h):
  alpha_h = 0.07*np.exp(-0.05*(v+70))
  beta_h =  1/(1 + np.exp(-0.1*(v+40)))

  h_inf = alpha_h/(alpha_h + beta_h)
  tau_h = 1/(alpha_h + beta_h)
 
  dh = (dt/tau_h)*(h_inf - h)
  return dh

# For each timestep we compute V and store the value
for t in range(len(time)-1):

  # For each timestep we compute n, m and h and store the value
  dn = compute_dn(V[t], n[t])
  n[t+1] = n[t] + dn

  dm = compute_dm(V[t], m[t])
  m[t+1] = m[t] + dm

  dh = compute_dh(V[t], h[t])
  h[t+1] = h[t] + dh

  # If time t >= 1 ms and t <= 2 ms, switch Injected Current ON
  if time[t] >= start_current and time[t]  <= end_current:
    ie = Ie
  # Otherwise, switch Injected Current OFF
  else:
    ie = 0

  # Use these values to compute the updated membrane current
  im[t+1] = GL*(V[t]-EL)+GK*np.power(n[t+1],4)*(V[t]-EK)+GNa*np.power(m[t+1],3)*h[t+1]*(V[t]-ENa)

  # Using Euler's Method for Numerical Integration (See Chapter Text)
  # we compute the change in voltage dV as follows (using the model equation)
  dV = dt*(-im[t+1] + ie/A)

  # Store this new value into our list
  V[t+1] = V[t] + dV

# Data 
x = list(time[0:-2])
y = V

# Plot data
fig = make_subplots(
    rows=3, cols=1,  shared_xaxes = True, vertical_spacing=0.1, 
    subplot_titles=('V over Time', 'i_m over Time', 'n, m, h over Time')
)

# Add traces
fig.add_trace(go.Scatter(name='V', x=x, y=y), row=1, col=1)
fig.add_trace(go.Scatter(name='i_m', x=x, y=im), row=2, col=1)
fig.add_trace(go.Scatter(name='n', x=x, y=n), row=3, col=1)
fig.add_trace(go.Scatter(name='m', x=x, y=m), row=3, col=1)
fig.add_trace(go.Scatter(name='h', x=x, y=h), row=3, col=1)

# Update xaxis properties
fig.update_xaxes(title_text="Time t (ms)", row=3, col=1)

# Update yaxis properties
fig.update_yaxes(title_text="Membrane Potential V (mV)", row=1, col=1)
fig.update_yaxes(title_text="Membrane Current i_m", row=2, col=1)
fig.update_yaxes(title_text="n, m, h (Probability)", row=3, col=1)

# Update title and size
fig.update_layout(height=800, width=700, 
                                title_text='Hodgkin-Huxley Model Neuron',
                                showlegend = True)

# Update theme
fig.layout.template = 'plotly_dark'

# Show figure
fig.show()

## Coding Challenge Problems

In [ ]:
#@title Run Simulation
#@markdown Execute the code block and use the sliders to set values in order to answer the Coding Challenge Problems in the chapter text.
#@markdown (Tip: Select a slider and use the left and right arrow keys to slide to the desired value.)
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets

# Voltage Parameters - Units mV (1 mV = 1e-3 Volts)
Vrest = -70

#Maximal Conductances - Units nS/mm^2
GL = 0.03
GK = 3.6
GNa = 12

# Reversal Potentials - Units mV
EL = -70
EK = -77
ENa = 55

# Input current: Ie - Units nA (1 nA = 10-9 Amperes)
Ie = 1.00

# Neuron Surface Area - Units mm^2
A = 0.1

# Simulation Time Span (0 to 20ms, dt = 0.01ms)
t0 = 0
dt = 0.01
t_final = 20
time = np.linspace(t0, t_final,  2000)

# Time at which the current is applied - Units ms
start_current = 1

# Time at which the current is switched off - Units ms
end_current = 2

# Initial voltage
Vinitial = Vrest

# Create a list V(t) to store the value of V at each time-step dt
V = [0] * len(time)

# Set the initial value at time t = t0 to the initial value Vinitial
V[0] =  Vinitial

# Initial gating variable values (Probability [0, 1])
n_initial = 0.1399
m_initial = 0.0498
h_initial = 0.6225

# Create lists to store the value of each gating variable at each time-step dt
n = [0] * len(time)
m= [0] * len(time)
h = [0] * len(time)

# Set the initial value at time t = t0 to the initial values
n[0] =  n_initial
m[0] =  m_initial
h[0] =  h_initial

# Initial membrane current 
im_initial = GL*(V[0]-EL)+GK*np.power(n[0],4)*(V[0]-EK)+GNa*np.power(m[0],3)*h[0]*(V[0]-ENa)

# Create list to store value of membrane current at each time-step dt
im = [0] * len(time)

# Set the initial value at time t = t0 to the initial value im_initial
im[0] = im_initial

# Function: compute_dn
def compute_dn(v, n):
  alpha_n = (0.01*(v + 60))/(1 - np.exp(-0.1*(v+60)))
  beta_n = 0.125*np.exp(-0.0125*(v+70))

  n_inf = alpha_n/(alpha_n + beta_n)
  tau_n = 1/(alpha_n + beta_n)

  dn = (dt/tau_n)*(n_inf - n)
  return dn

# Function: compute_dm
def compute_dm(v, m):
  alpha_m = (0.1*(v + 45))/(1 - np.exp(-0.1*(v+45)))
  beta_m = 4*np.exp(-0.0556*(v+70))

  m_inf = alpha_m/(alpha_m + beta_m)
  tau_m = 1/(alpha_m + beta_m)

  dm = (dt/tau_m)*(m_inf - m)
  return dm

# Function: compute_dh
def compute_dh(v, h):
  alpha_h = 0.07*np.exp(-0.05*(v+70))
  beta_h =  1/(1 + np.exp(-0.1*(v+40)))

  h_inf = alpha_h/(alpha_h + beta_h)
  tau_h = 1/(alpha_h + beta_h)
 
  dh = (dt/tau_h)*(h_inf - h)
  return dh

def compute_iaf_neuron(Ie):
  # For each timestep we compute V and store the value
  for t in range(len(time)-1):

    # For each timestep we compute n, m and h and store the value
    dn = compute_dn(V[t], n[t])
    n[t+1] = n[t] + dn

    dm = compute_dm(V[t], m[t])
    m[t+1] = m[t] + dm

    dh = compute_dh(V[t], h[t])
    h[t+1] = h[t] + dh

    # If time t >= 1 ms and t <= 2 ms, switch Injected Current ON
    if time[t] >= start_current and time[t]  <= end_current:
      ie = Ie
    # Otherwise, switch Injected Current OFF
    else:
      ie = 0

    # Use these values to compute the updated membrane current
    im[t+1] = GL*(V[t]-EL)+GK*np.power(n[t+1],4)*(V[t]-EK)+GNa*np.power(m[t+1],3)*h[t+1]*(V[t]-ENa)

    # Using Euler's Method for Numerical Integration (See Chapter Text)
    # we compute the change in voltage dV as follows (using the model equation)
    dV = dt*(-im[t+1] + ie/A)

    # Store this new value into our list
    V[t+1] = V[t] + dV
  return V

def plot_iaf_neuron(V, time):
  # Data 
  x = list(time[0:-2])
  y = V

  # Plot data
  fig = make_subplots(
      rows=3, cols=1,  shared_xaxes = True, vertical_spacing=0.1, 
      subplot_titles=('V over Time', 'i_m over Time', 'n, m, h over Time')
  )

  # Add traces
  fig.add_trace(go.Scatter(name='V', x=x, y=y), row=1, col=1)
  fig.add_trace(go.Scatter(name='i_m', x=x, y=im), row=2, col=1)
  fig.add_trace(go.Scatter(name='n', x=x, y=n), row=3, col=1)
  fig.add_trace(go.Scatter(name='m', x=x, y=m), row=3, col=1)
  fig.add_trace(go.Scatter(name='h', x=x, y=h), row=3, col=1)

  # Update xaxis properties
  fig.update_xaxes(title_text="Time t (ms)", row=3, col=1)

  # Update yaxis properties
  fig.update_yaxes(title_text="Membrane Potential V (mV)", row=1, col=1)
  fig.update_yaxes(title_text="Membrane Current i_m", row=2, col=1)
  fig.update_yaxes(title_text="n, m, h (Probability)", row=3, col=1)

  # Update title and size
  fig.update_layout(height=800, width=700, 
                                  title_text='Hodgkin-Huxley Model Neuron',
                                  showlegend = True)

  # Update theme
  fig.layout.template = 'plotly_dark'

  # Show figure
  fig.show()

style = {'description_width':'auto'}
@widgets.interact(
  Ie = widgets.FloatSlider(
    value=1.00,
    min=1.00,
    max=2.00,
    step=0.1,
   description='Ie',
    style = style,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='1.2f'
  )
)
def simulate_iaf_neuron(Ie=1.00):
  V = compute_iaf_neuron(Ie)
  plot_iaf_neuron(V, time)